In [12]:
# Make notebook full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Create IDF file with all output variables requested and runs model

File Name: PyEP_ALL_VARIABLES
Author: Anjukan Kathirgamanathan
Date: 26/08/2018

This is a python script to create an IDF file that requests all possible output variables, runs it, and exports the data to a pandas dataframe.

## User Input

In [13]:
# Specify the run period
begin_month = 1
begin_month_day = 1
end_month = 12
end_month_day = 31
start_day = "Sunday"

# Specify path to idd file
iddfile = "V8-7-0-Energy+.idd"

# Set the path to the original (REF) IDF file
idf_path = "RefBldgLargeOfficeNew2004_v1.4_7.2_4C_Ref.idf"

# Set the path to the .rdd file
rdd_path = "parametric/ref_TES/RefBldgLargeOfficeNew2004_v1.4_7.2_4C_Ref.rdd"

# Set the path to the file with the keep status of the variables (optional if some variables are to be omitted)
keepfile = 'ALL_variables_request.csv'

# Name of the new IDF created with all variables requested
new_idf = "RefBldgLargeOfficeNew2004_v1.4_7.2_4C_Ref_ALL.idf"

# Paths and file names for simulation run (weather file, output directory and prefix for output)
weather_path = "IRL_Dublin.039690_IWEC.epw"
out_path = "ALL"
out_prefix = "RefBldgLargeOfficeNew2004_v1.4_7.2_4C_Ref_ALL"


### Load required libraries


In [14]:
# Eppy (https://pypi.org/project/eppy/ by Santhosh Philip)
from eppy import modeleditor 
from eppy.modeleditor import IDF

import numpy

import pandas as pd

import glob
import os
import json

## Read in files and variables

eppy requires that we load the `.idd` from EnergyPlus and the `.idf` that we want to manipulate

In [15]:
# Set the path to the idd file
IDF.setiddname(iddfile)

Let's load the IDF file where we want to make the changes and request all output variables. 
Let's also load the .rdd file from which we can obtain a list of all the output variables that can be requested.

In [16]:
# Read the idf file
ref1 = IDF(idf_path)

# Open the .rdd file for all output variable names to be read in
text_file = open(rdd_path, "r")
# Split the lines such that commas and the '[' are used as delimiters
import re
lines = text_file.read()
import_all = re.split("[[,\!?:]", lines)

# Delete the first few strings from the prelog
del import_all[0:10];
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

# Keep every 3rd element as this represents the output variable
variables = import_all[0::3]
print(variables)

# See how many output variables there are in total
total_variables = len(variables)
print(total_variables)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
['Site Outdoor Air Drybulb Temperature ', 'Site Outdoor Air Dewpoint Temperature ', 'Site Outdoor Air Wetbulb Temperature ', 'Site Outdoor Air Humidity Ratio ', 'Site Outdoor Air Relative Humidity ', 'Site Outdoor Air Barometric Pressure ', 'Site Wind Speed ', 'Site Wind Direction ', 'Site Sky Temperature ', 'Site Horizontal Infrared Radiation Rate per Area ', 'Site Diffuse Solar Radiation Rate per Area ', 'Site Direct Solar Radiation Rate per Area ', 'Site Precipitation Depth ', 'Site Ground Reflected Solar Radiation Rate per Area ', 'Site Ground Temperature ', 'Site Surface Ground Temperature ', 'Site Deep Ground Temperature ', 'Site Simple Factor Model Ground Temperature ', 'Site Outdoor Air Enthalpy ', 'Site Outdoor Air Density ', 'Site Solar Azimuth Angle ', 'Site Solar Altitude Angle ', 'Site Solar Hour Angle ', 'Site Rain Status ', 'Site Snow on Ground Status ', 'Site Exterior Horizontal Sky 

A first filtering is performed to remove non observable variables (and to decrease the size of the initial dataframe). The keep status of each variable is taken from the 'ALL_variables_request.csv' file.

In [17]:
# Import in keep status of variables
keep = pd.read_csv(keepfile)

# Filter to be all the variables to remove
keep = keep[keep.Keep != 1]
sep = '['
keep.Variable = keep.Variable.str.split('[').str[0]

# Convert from a series to a list
to_remove = keep.Variable.tolist()

# Remove variables that are in the to_remove list
variables = [item for item in variables if item not in to_remove]

# See how many output variables there are in total
total_variables = len(variables)
print(total_variables)

228


There are 284 variables in total after filtering. Let's see all of the existing `OUTPUT:VARIABLE` objects. We will have to delete the existing variables first to ensure no variables are repeated.

In [18]:
# Clear any existing output variables requested
output_variables = ref1.idfobjects['OUTPUT:VARIABLE']
print(output_variables)

output_variables.clear()

[
Output:Variable,
    *,                        !- Key Value
    Erl Chiller Outlet Setpoint,    !- Variable Name
    Timestep;                 !- Reporting Frequency
, 
Output:Variable,
    *,                        !- Key Value
    Plant Supply Side Cooling Demand Rate,    !- Variable Name
    Timestep;                 !- Reporting Frequency
, 
Output:Variable,
    *,                        !- Key Value
    Facility Thermal Comfort ASHRAE 55 Simple Model Summer Clothes Not Comfortable Time,    !- Variable Name
    Timestep;                 !- Reporting Frequency
, 
Output:Variable,
    *,                        !- Key Value
    Chilled Water Thermal Storage Tank Temperature,    !- Variable Name
    Timestep;                 !- Reporting Frequency
, 
Output:Variable,
    *,                        !- Key Value
    Chilled Water Thermal Storage Use Side Mass Flow Rate,    !- Variable Name
    Timestep;                 !- Reporting Frequency
, 
Output:Variable,
    *,                   

We need to change the code for "Output:Variable" class to add objects

### Add new objects in a loop

In [19]:
# Set up a numpy array with the total number of variables
variables_manual = numpy.linspace(0, total_variables-1, num=total_variables)

# In a for loop, add all the output variables
for i in variables_manual:
    
    # Add new object
    ref1.newidfobject("OUTPUT:VARIABLE")
    output_variables[-1].Variable_Name = variables[int(i)]
    output_variables[-1].Reporting_Frequency = "Timestep"
    #print(output_variables)

## Modify run period

In [20]:
# Make changes to the RunPeriod Object based on user input above
ref1.idfobjects['RUNPERIOD']

Run = ref1.idfobjects["RUNPERIOD"][0]

Run.Begin_Month = begin_month
Run.Begin_Day_of_Month = begin_month_day
Run.End_Month = end_month
Run.End_Day_of_Month = end_month_day
Run.Day_of_Week_for_Start_Day = start_day

ref1.idfobjects['RUNPERIOD']

[
RunPeriod,
    ,                         !- Name
    1,                        !- Begin Month
    1,                        !- Begin Day of Month
    12,                       !- End Month
    31,                       !- End Day of Month
    Sunday,                   !- Day of Week for Start Day
    No,                       !- Use Weather File Holidays and Special Days
    No,                       !- Use Weather File Daylight Saving Period
    No,                       !- Apply Weekend Holiday Rule
    Yes,                      !- Use Weather File Rain Indicators
    Yes,                      !- Use Weather File Snow Indicators
    1.0000;                   !- Number of Times Runperiod to be Repeated
]

### Save the new IDF file

In [21]:
print("Creating IDF file with "+ str(total_variables)+" Output Variables")

# Save the IDF file
ref1.saveas(new_idf)

Creating IDF file with 228 Output Variables


## Run the IDF file

In [22]:
# We first need to define the file locations
# Note we are using V8.70 EP+
ep_cmd = "C:\EnergyPlusV8-7-0\energyplus -x -r -i " + iddfile + " -w " + weather_path + " -d " + out_path + " -p " + out_prefix + " -s C " + new_idf
ep_cmd

# now we use ! to call command lines in IPython
# this may take a while
# if simulation is successful, you will see a bunch of command line outputs from EnergyPlus
!$ep_cmd

ExpandObjects Started.
No expanded file generated.
ExpandObjects Finished. Time:     0.078
EnergyPlus Starting
EnergyPlus, Version 8.7.0-78a111df4a, YMD=2018.08.26 21:21
Processing Data Dictionary
Processing Input File
Initializing Response Factors
Calculating CTFs for "MASS NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "UNDERGROUND WALL NON-RES", Construction # 7
Calculating CTFs for "DROPCEILING", Construction # 9
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 11
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing

EnergyPlus Completed Successfully.
